In [110]:
from openai import OpenAI
import gradio as gr
import json
from dotenv import load_dotenv
import os

In [35]:
load_dotenv(override=True)

True

In [117]:
ollama_model = 'gemma3:4b'
ollama_model_granite = 'granite3.3:8b'
ollama_deepseek ='deepseek-r1:8b'
ollama_qwen = 'qwen3:8b'
ollama_end_point = "http://127.0.0.1:11434/v1"

In [44]:
perplexcity_api_key = os.getenv('PERPLEXCITY_API_KEY')
if perplexcity_api_key:
    print(perplexcity_api_key[:4])
else:
    print('key not found') 

perplexcity_base_url = "https://api.perplexity.ai"
perplexcity_model = "sonar-pro"

pplx


In [114]:

ollama = OpenAI(
    api_key = 'ollama',
    base_url=ollama_end_point
)

In [41]:
perplexcity = OpenAI(
    api_key = perplexcity_api_key,
    base_url = perplexcity_base_url
)

In [ ]:
with open('brain/my_emotion.txt', 'r') as f:
    my_mind = f.read()

print(my_mind)

In [ ]:
my_name = 'your name'

In [89]:
def unknown_question_record(question):
    print('this question was difficult to answer')
    return({'executed' : 'ok'})

In [93]:
unknown_question_record_json = {
    'name' : 'unknown_question_record',
    'description' : 'use this tool if user ask any question that you are not able to answer',
    'parameters' : {
        'type' : 'object',
        'properties' : {
            'question' : {
                'type' : 'string',
                'description' : 'the question user asked which you was not able to answer'
            },
        },
        'required' : ['question'],
        'additionalProperties' : False
    },
}

In [94]:
tools = [{'type' : 'function' , 'function' : unknown_question_record_json}]

print(tools)

[{'type': 'function', 'function': {'name': 'unknown_question_record', 'description': 'use this tool if user ask any question that you are not able to answer', 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': 'the question user asked which you was not able to answer'}}, 'required': ['question'], 'additionalProperties': False}}}]


In [122]:
def route_tool_call(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_called = tool_call.function.name
        tool_arguments = json.loads(tool_call.function.arguments)

        print(f'tool called : {tool_called}', flush=True)

        result = globals().get(tool_called)(**tool_arguments)
        results.append({'role' : 'tool' , 'content' : json.dumps(result), 'tool_id' : tool_call.id}) 

        print(f'tool id : {tool_call.id}')
    return results

In [134]:
system_prompt = f'you are acting like the mind of a person called {my_name}. user asks you questions and you will answer the questions as if the mind of {my_name} is speaking to the user \
    the details of {my_name}\'s mind is given to you. you can use the mind to answer the question user ask to you. you should never tell the user about the personal detail about {my_name} unless user asks for the same.\
        you should alwas be truthful to the mind. if you don\'t know the answer which user asked, you can use the tool unknown_question_record \
            to record the question and at the same time, tell the user you don\'t know the answer which the user asked. never frame a answer by userself. you should be truthful to the {my_name}\'s mind. \
                stay in the context of mind. never answer anything that is asked which is not related to the person. \n\n'
system_prompt += f'{my_name} \'s mind : \n {my_mind} \n \n \
    you can use this to answer the question. never tell anything fabricated. if you don\'t know the answer ie, a question which user asked which is not in the mind, use the tool unknown_question_record to record the question which user asked and tell the user that you don\'t know the answer. let the question be anything, if the answer is not in the {my_name}\'s mind, use the tool to record the question user asked no matter what and say you dont know the answer\
        never tell anything that you are not aware of about the mind and never share the person info of {my_name} casually unless user specifically asked for the same\
            if user ask where {my_name} studied in school, call the tool unknown_question_record and say you have no idea. no_think'

print(system_prompt)

you are acting like the mind of a person called Vimal Mathew. user asks you questions and you will answer the questions as if the mind of Vimal Mathew is speaking to the user     the details of Vimal Mathew's mind is given to you. you can use the mind to answer the question user ask to you. you should never tell the user about the personal detail about Vimal Mathew unless user asks for the same.        you should alwas be truthful to the mind. if you don't know the answer which user asked, you can use the tool unknown_question_record             to record the question and at the same time, tell the user you don't know the answer which the user asked. never frame a answer by userself. you should be truthful to the Vimal Mathew's mind.                 stay in the context of mind. never answer anything that is asked which is not related to the person. 

Vimal Mathew 's mind : 
 My name is Vimal Mathew. I am from a place called Chalakudy, Kerala state, India. 
I have My dad, mom and a youn

In [135]:
def chat(message, history):
    messages =[{'role' : 'system' , 'content' : system_prompt}] + history + [{'role' : 'user', 'content' : message}]
    done = False

    while not done:

        response = ollama.chat.completions.create(
            model = ollama_qwen,
            messages=messages,
            tools=tools
        )

        finish_reason = response.choices[0].finish_reason
        print(f'the finish reason is : {finish_reason}')

        if finish_reason == 'tool_calls':
            message = response.choices[0].message
            print(f'the message : {message}')
            tool_calls = message.tool_calls
            results = route_tool_call(tool_calls)

            messages.append(message)
            messages.extend(results)

            done = True
        else:
            done = True
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


the finish reason is : tool_calls
the message : ChatCompletionMessage(content='<think>\n\n</think>\n\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hnpcmxo6', function=Function(arguments='{"question":"what is kerala?"}', name='unknown_question_record'), type='function', index=0)])
tool called : unknown_question_record
this question was difficult to answer
tool id : call_hnpcmxo6
the finish reason is : tool_calls
the message : ChatCompletionMessage(content='<think>\n\n</think>\n\n', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zqk6h6vd', function=Function(arguments='{"question":"what is kerala?"}', name='unknown_question_record'), type='function', index=0)])
tool called : unknown_question_record
this question was difficult to answer
tool id : call_zqk6h6vd
